In [3]:
import cv2
import time
import os

# Load the cascade classifiers for face and eye detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# Start the video capture
capture = cv2.VideoCapture(0)

# Create a folder to save images if it doesn't exist
save_path = 'saved_eyes/'
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Variable to track the time and save the image every minute
start_time = time.time()  # Start the timer
last_saved_time = start_time  # Track the last time an image was saved

# Initialize counter for eye image naming
image_counter = 0

while True:
    # Read the frame from the video capture
    ret, frame = capture.read()

    # Check if the frame was read successfully
    if not ret:
        print("Error: Could not read frame from video capture.")
        break  # Exit the loop if frame reading fails

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Loop over the faces
    for (x, y, w, h) in faces:
        # Get the face ROI
        face_roi = gray[y:y+h, x:x+w]
        face_roi_color = frame[y:y+h, x:x+w]

        # Detect eyes in the face ROI
        eyes = eye_cascade.detectMultiScale(face_roi)

        # Loop over the eyes
        for i, (ex, ey, ew, eh) in enumerate(eyes):
            # Extract the eye region from the face
            eye_region = face_roi_color[ey:ey+eh, ex:ex+ew]

            # Resize the eye region for the redness detection model input (e.g., 224x224)
            eye_resized = cv2.resize(eye_region, (224, 224))

            # Optional: Convert the image to RGB if required for the model
            eye_resized_rgb = cv2.cvtColor(eye_resized, cv2.COLOR_BGR2RGB)

            # Save each eye image separately
            if i == 0:  # For the first eye (left eye)
                cv2.imwrite(f"{save_path}eye_left_{image_counter}.jpg", eye_resized)  # Save the left eye
            elif i == 1:  # For the second eye (right eye)
                cv2.imwrite(f"{save_path}eye_right_{image_counter}.jpg", eye_resized)  # Save the right eye

    # Get the current time
    current_time = time.time()

    # Save the eye images every 1 minute
    if current_time - last_saved_time >= 20:  # Save every 1 minute
        image_counter += 1  # Increment the image counter
        last_saved_time = current_time  # Update the last saved time

    # Display the frame with rectangles around the face and eyes (optional)
    cv2.imshow('Face and Eye Detection', frame)

    # Check if the user pressed 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and destroy the windows
capture.release()
cv2.destroyAllWindows()
